In [ ]:
import json
import numpy as np

# Your JSON data
json_file_path = "./data/maincrops.json"

# Open and read the JSON file
with open(json_file_path, "r") as json_file:
    # Parse the JSON data into a Python dictionary
    data = json.load(json_file)

# # Function to calculate average and standard deviation
def calculate_stats(values):
    avg = np.mean(values)
    std = np.std(values)
    return avg, std

# # Iterate through each crop
for crop_key, crop_data in data.items():
    ertrag_data = crop_data["Ertrag"]
    
    # Iterate through each region under "Ertrag"
    for region_key, region_data in ertrag_data.items():
        values = region_data["Values"]
        
        # Calculate average and standard deviation
        avg, std = calculate_stats(values)
        
        # Add "avg" and "std" to the region_data
        region_data["avg"] = avg
        region_data["std"] = std

# # Convert the updated data back to JSON format
updated_json = json.dumps(data, indent=2)

# # Print the updated JSON data
print(updated_json)

In [13]:
#### Append GBM parameters


import numpy as np
import json
import numpy as np

# Your JSON data
json_file_path = "./data/maincrops.json"

# Open and read the JSON file
with open(json_file_path, "r") as json_file:
    # Parse the JSON data into a Python dictionary
    data = json.load(json_file)

# # Function to calculate average and standard deviation
def calculate_stats(values):
    avg = np.mean(values)
    std = np.std(values)
    return avg, std


# # # Function to calculate average and standard deviation
def calculate_stats_gbm(values):
    avg = np.mean(values)
    std = np.std(values)

    returns = np.diff(values) / values[:-1]

    # Calculate the average annual return (drift)
    avg_return = np.mean(returns)

    # Calculate annual volatility (sigma)
    volatility = np.std(returns)
    return avg, std, avg_return, volatility



kosten_keys = ["Saatgut","Dünger","Sonstiges"]
# # # Iterate through each crop
for crop_key, crop_data in data.items():
    print(crop_key)

    # Ertrag-Stats
    ertrag_data = crop_data["Ertrag"]
    
    # Iterate through each region under "Ertrag"
    for region_key, region_data in ertrag_data.items():
        values = region_data["Values"]
        
        # Calculate average and standard deviation
        avg, std = calculate_stats(values)
        
        # Add "avg" and "std" to the region_data
        region_data["avg"] = avg
        region_data["std"] = std

    # Verkaufspreis GBM Stats
    verkaufspreis = crop_data["Verkaufspreis"]
    values = region_data["Values"]
        
    # Calculate average and standard deviation
    avg, std, avg_return, volatility = calculate_stats_gbm(values)
        
    # Add "avg" and "std" to the region_data
    verkaufspreis["avg"] = avg
    verkaufspreis["std"] = std
    verkaufspreis["avg_gbm"] = avg_return
    verkaufspreis["std_gbm"] = volatility


    # Kosten GBM Stats
    kosten = crop_data["Kosten"]
    for key in kosten_keys:
        values = kosten[key]["Values"]
        avg, std, avg_return, volatility = calculate_stats_gbm(values)
        kosten[key]["avg"] = avg
        kosten[key]["std"] = std
        kosten[key]["avg_gbm"] = avg_return
        kosten[key]["std_gbm"] = volatility

# Dump the dictionary to a JSON file with ensure_ascii=False
with open("./data/maincrops_updated.json", "w", encoding="utf-8") as json_file:
    json.dump(data, json_file, ensure_ascii=False)

KLEEGRAS
LUZERNE
ACKERBOHNE
KÖRNERERBSE
SOJABOHNE
FUTTERLUPINE
WINTERWEIZEN
SOMMERWEIZEN
WINTERHARTWEIZEN
WINTERDINKEL
WINTERTRITICALE
WINTERROGGEN
WINTERGERSTE
SOMMERGERSTE
SOMMERHAFER
HIRSE
SILOMAIS
KÖRNERMAIS
ZUCKERRÜBEN
KARTOFFELN
WINTERRAPS
SONNENBLUMEN
ÖLKÜRBIS
{
  "KLEEGRAS": {
    "Crop": "Bio-Kleegras 2J",
    "Years": [
      0,
      1,
      2,
      3,
      4,
      5,
      6,
      7
    ],
    "cropTypes": [
      "legume",
      "kleinlegume",
      "gras"
    ],
    "season": "winter",
    "Ertrag": {
      "Bundesgebiet": {
        "Unit": "t/ha",
        "Values": [
          24.5,
          24.5,
          24.5,
          24.5,
          24.5,
          24.5,
          24.5,
          24.5
        ],
        "avg": 24.5,
        "std": 0.0
      }
    },
    "Verkaufspreis": {
      "Unit": "\u20ac/t",
      "Values": [
        67.1,
        67.1,
        62.8,
        66.7,
        70.6,
        81.3,
        65.7,
        69.5
      ],
      "avg": 24.5,
      "

c:\Users\Magnus\anaconda3\envs\cro_symbolic_mbpo_env\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\Magnus\anaconda3\envs\cro_symbolic_mbpo_env\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\Magnus\anaconda3\envs\cro_symbolic_mbpo_env\lib\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\Magnus\anaconda3\envs\cro_symbolic_mbpo_env\lib\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
c:\Users\Magnus\anaconda3\envs\cro_symbolic_mbpo_env\lib\site-packages\numpy\core\_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
